## Scénario de recherche : récupération des métadonnées et des OCR (API SRU).
---

**Référence** : Guillaume Pressiat (CHRU Brest)

   - Github : [https://github.com/GuillaumePressiat/gargallica](https://github.com/GuillaumePressiat/gargallica)

---


### Installation d'un noyau R dans un jupyter notebook :

   - [https://docs.anaconda.com/anaconda/navigator/tutorials/r-lang/](https://docs.anaconda.com/anaconda/navigator/tutorials/r-lang/)

---

### Faire une recherche avancée sur gallica.bnf.fr

Par exemple : 

- Je cherche tout ce qu'il y a comme réponse sur 

    - l'auteur Guy de Maupassant
    - en monographies
    - disponible sur Gallica : [https://gallica.bnf.fr/](https://gallica.bnf.fr/)
    - libre de droits

</br>

- Je copie le lien internet 

</br>

- https://gallica.bnf.fr/services/engine/search/sru?operation=searchRetrieve&exactSearch=false&collapsing=true&version=1.2&query=(dc.creator%20all%20%22Maupassant,%20Guy%20de%22%20or%20dc.contributor%20all%20%22Maupassant,%20Guy%20de%22%20)%20%20and%20(dc.type%20all%20%22monographie%22)%20and%20(provenance%20adj%20%22bnf.fr%22)%20and%20(access%20all%20%22fayes%22)&suggest=10&keywords=Maupassant,%20Guy%20de

</br>

- Et je récupère tout ce qui suit query= 

</br>

- (dc.creator%20all%20%22Maupassant,%20Guy%20de%22%20or%20dc.contributor%20all%20%22Maupassant,%20Guy%20de%22%20)%20%20and%20(dc.type%20all%20%22monographie%22)%20and%20(provenance%20adj%20%22bnf.fr%22)%20and%20(access%20all%20%22fayes%22)&suggest=10&keywords=Maupassant,%20Guy%20de

### Pourquoi passer par la barre de recherche de Gallica plutôt que d'en rester dans à requête au sein du Jupyter notebook ?


---

In [1]:
# Copie de la query :

query = '(dc.creator%20all%20%22Maupassant,%20Guy%20de%22%20or%20dc.contributor%20all%20%22Maupassant,%20Guy%20de%22%20)%20%20and%20(dc.type%20all%20%22monographie%22)%20and%20(provenance%20adj%20%22bnf.fr%22)%20and%20(access%20all%20%22fayes%22)&suggest=10&keywords=Maupassant,%20Guy%20de'

# Amorce de récupération du XML :

i = 1

requete <- function(i)xml2::read_xml(paste0(
    'http://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=(', 
    query,')&collapsing=false&maximumRecords=50&startRecord=',
    i))

In [2]:
# Première 50 réponses (initialiser la structure xml avec un premier coup)
flux_xml <- requete(1)
flux_xml

{xml_document}
<searchRetrieveResponse xmlns:ns5="http://gallica.bnf.fr/namespaces/gallica/" xmlns:oai_dc="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:srw="http://www.loc.gov/zing/srw/" xmlns:dc="http://purl.org/dc/elements/1.1/">
[1] <srw:version>1.2</srw:version>
[2] <srw:echoedSearchRetrieveRequest>\n  <srw:query>((dc.creator all "Maupass ...
[3] <srw:numberOfRecords>121</srw:numberOfRecords>
[4] <srw:extraResponseData>&lt;numberOfRecordsDecollapser&gt;121&lt;/numberOf ...
[5] <srw:records>\n  <srw:record>\n    <srw:recordSchema>http://www.openarchi ...
[6] <srw:nextRecordPosition>51</srw:nextRecordPosition>

In [3]:
# récupérer le nombre total de réponses
xml_list <- xml2::as_list(flux_xml)
nb_rep <- as.integer(unlist(xml_list$searchRetrieveResponse$numberOfRecords))
nb_rep

[1] 121

In [4]:
# Boucle sur la suite, 50 par 50
# Ajouter au document xml tot les réponses des autres pages
for (j in seq(51, nb_rep, by = 50)){
  temp <- requete(j)
  for (l in xml2::xml_children(temp)){
    xml2::xml_add_child(flux_xml, l)
  }
}

In [5]:
flux_xml

{xml_document}
<searchRetrieveResponse xmlns:ns5="http://gallica.bnf.fr/namespaces/gallica/" xmlns:oai_dc="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:srw="http://www.loc.gov/zing/srw/" xmlns:dc="http://purl.org/dc/elements/1.1/">
 [1] <srw:version>1.2</srw:version>
 [2] <srw:echoedSearchRetrieveRequest>\n  <srw:query>((dc.creator all "Maupas ...
 [3] <srw:numberOfRecords>121</srw:numberOfRecords>
 [4] <srw:extraResponseData>&lt;numberOfRecordsDecollapser&gt;121&lt;/numberO ...
 [5] <srw:records>\n  <srw:record>\n    <srw:recordSchema>http://www.openarch ...
 [6] <srw:nextRecordPosition>51</srw:nextRecordPosition>
 [7] <srw:version xmlns:srw="http://www.loc.gov/zing/srw/">1.2</srw:version>
 [8] <srw:echoedSearchRetrieveRequest xmlns:srw="http://www.loc.gov/zing/srw/ ...
 [9] <srw:numberOfRecords xmlns:srw="http://www.loc.gov/zing/srw/">121</srw:n ...
[10] <srw:extraResponseData xmlns:srw="http://www.loc.gov/zing/srw/">&lt;numb ...
[11] <srw:records xmlns:srw="http://www.loc.gov/

In [6]:
# Sauvegarde du flux XML :

xml2::write_xml(flux_xml, 'Maupassant.xml')

NULL

In [7]:
# Transformation du XML en dataframe (tableau) : 

library('xml2')
library('purrr')
library('dplyr')
library('tidyr')

xml_to_df <- function(doc, ns = xml_ns(doc)) {
  split_by <- function(.x, .f, ...) {
    vals <- map(.x, .f, ...)
    split(.x, simplify_all(transpose(vals)))
  }
  node_to_df <- function(node) {
    # Filter the attributes for ones that aren't namespaces
    # x <- list(.index = 0, .name = xml_name(node, ns))
    x <- list(.name = xml_name(node, ns))
    # Attributes as column headers, and their values in the first row
    attrs <- xml_attrs(node)
    if (length(attrs) > 0) {attrs <- attrs[!grepl("xmlns", names(attrs))]}
    if (length(attrs) > 0) {x <- c(x, attrs)}
    # Build data frame manually, to avoid as.data.frame's good intentions
    children <- xml_children(node)
    if (length(children) >= 1) {
      x <- 
        children %>%
        # Recurse here
        map(node_to_df) %>%
        split_by(".name") %>%
        map(bind_rows) %>%
        map(list) %>%
        {c(x, .)}
      attr(x, "row.names") <- 1L
      class(x) <- c("tbl_df", "data.frame")
    } else {
      x$.value <- xml_text(node)
    }
    x
  }
  node_to_df(doc)
}

parse_gallica <- function(x){
  xml2::xml_find_all(flux_xml, ".//srw:recordData")[x] %>% 
    xml_to_df() %>% # application de la fonction précédente
    select(-.name) %>% 
    .$`oai_dc:dc` %>% 
    .[[1]] %>% 
    mutate(recordId = 1:nrow(.)) %>% 
#    tidyr::unnest() %>% 
    tidyr::gather(var, val, - recordId) %>% 
    group_by(recordId, var) %>% 
    mutate(value = purrr::map(val, '.value') %>% purrr::flatten_chr() %>% paste0( collapse = " -- ")) %>% 
    select(recordId, var, value) %>% 
    ungroup() %>% 
    mutate(var = stringr::str_remove(var, 'dc:')) %>% 
    tidyr::spread(var, value) %>% 
    select(-.name)
}

tot_df <- 1:nb_rep %>% 
  parse_gallica %>% 
  bind_rows()

head(tot_df)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



recordId,contributor,creator,date,description,format,identifier,language,publisher,relation,rights,source,subject,title,type
1,"Lepère, Auguste (1849-1918). Illustrateur","Maupassant, Guy de (1850-1893). Auteur du texte -- Maupassant, Guy de (1850-1893). Auteur du texte",1907,Comprend : La ficelle -- Appartient à l’ensemble documentaire : BNormand1 -- Appartient à l’ensemble documentaire : HNormand1,1 vol. (89 p.) : fig. en coul. ; in-8 -- Nombre total de vues : 129,https://gallica.bnf.fr/ark:/12148/bpt6k1049488s,fre,(Paris),Notice du catalogue : http://catalogue.bnf.fr/ark:/12148/cb30910533g,domaine public -- public domain,"Bibliothèque nationale de France, département Réserve des livres rares, RES P-Y2-445",,Deux contes / Maupassant ; 84 petites compositions dessinées et gravées sur bois par Auguste Lepère,text -- monographie imprimée -- printed monograph
2,,"Maupassant, Guy de (1850-1893). Auteur du texte",1882,Contient une table des matières -- Avec mode texte,1 vol. (172 p.) : portrait ; in-16 -- Nombre total de vues : 186,https://gallica.bnf.fr/ark:/12148/bpt6k5680846d,fre -- français,H. Kistemaeckers (Bruxelles),Notice du catalogue : http://catalogue.bnf.fr/ark:/12148/cb324319162,domaine public -- public domain,"Bibliothèque nationale de France, département Réserve des livres rares, RES P-Y2-2092",,Mlle Fifi / Guy de Maupassant ; eau-forte par Just,text -- monographie imprimée -- monographie imprimée
3,,"Maupassant, Guy de (1850-1893). Auteur du texte",1884,Contient une table des matières -- Avec mode texte,III-348 p. ; in-18 -- Nombre total de vues : 362 -- application/epub+zip,https://gallica.bnf.fr/ark:/12148/bpt6k58416576,fre -- français,V. Havard (Paris),Notice du catalogue : http://catalogue.bnf.fr/ark:/12148/cb30910585q,domaine public -- public domain,"Bibliothèque nationale de France, département Littérature et art, 8-Y2-6879",,Miss Harriet / Guy de Maupassant,text -- monographie imprimée -- monographie imprimée
4,"Normandy, Georges (1882-1946). Éditeur scientifique","Maupassant, Guy de (1850-1893). Auteur du texte",1913,Contient une table des matières -- Avec mode texte,"1 vol. (187 p.) : portr., fac-sim., pl. ; in-16 -- Nombre total de vues : 196",https://gallica.bnf.fr/ark:/12148/bpt6k5801596v,fre -- français,A. Méricant (Paris),Notice du catalogue : http://catalogue.bnf.fr/ark:/12148/cb365719754,domaine public -- public domain,"Bibliothèque nationale de France, département Littérature et art, Z BARRES-22799",,"Anthologie des auteurs modernes. V, Recueil de morceaux choisis / Guy de Maupassant ; précédé d'une étude... par Georges Normandy...",text -- monographie imprimée -- monographie imprimée
5,,"Maupassant, Guy de (1850-1893). Auteur du texte",1883,[Contes de la bécasse (français)] -- Avec mode texte,"1 vol. (298 p.) ; in-16 -- Nombre total de vues : 313 -- application/epub+zip 3.0 accessible -- Format adaptable de type XML DTBook, 2005-3",https://gallica.bnf.fr/ark:/12148/bpt6k10407406,fre -- français,E. Rouveyre et G. Blond (Paris),Notice d’oeuvre : http://catalogue.bnf.fr/ark:/12148/cb12376516m -- Notice du catalogue : http://catalogue.bnf.fr/ark:/12148/cb324318906,domaine public -- public domain,"Bibliothèque nationale de France, département Réserve des livres rares, RES 8-Z DON-595 (61)",,Contes de la bécasse. 2è édition (2e édition) / Guy de Maupassant,text -- monographie imprimée -- monographie imprimée
6,,"Maupassant, Guy de (1850-1893). Auteur du texte",1884,Avec mode texte,"1 vol. (145 p.) : fig., pl. en divers états ; in-4 -- Nombre total de vues : 209 -- application/epub+zip 3.0 accessible -- Format adaptable de type XML DTBook, 2005-3",https://gallica.bnf.fr/ark:/12148/bpt6k10407688,fre -- français,E. Monnier (Paris),Notice du catalogue : http://catalogue.bnf.fr/ark:/12148/cb309105088,domaine public -- public domain,"Bibliothèque nationale de France, département Réserve des livres rares, SMITH LESOUEF R-6505",,Clair de lune / Guy de Maupassant,text -- monographie imprimée -- monographie imprim

In [8]:
# Calcul nqa

# Cf. https://www.bnf.fr/sites/default/files/2018-11/referentiels_num_ocr.pdf
## Section 7.2.3 :

    # Dans le manifeste du document numérique (refNum ou METS) 
    # Les indicateurs à fournir (à l’échelle du document) sont notamment : 

        # le taux OCR brut :  le    taux de confiance en sortie de l’OCR
        # le taux OCR corrigé :  le    taux de confiance après correction manuelle
        # le coefficient d’étalonnage utilisé (cf. section 7.2.2)
        # le   taux   NQA   moyen,   exprimé   par : taux   OCR   corrigé   × coefficient d’étalonnage (= moyenne de qualité de toutes les pages)

parse_extraRecordData <- function(x){
xml2::xml_find_all(flux_xml, ".//srw:extraRecordData")[x] %>% 
  xml_to_df() %>% 
  select(-.name) %>% 
  #select(`d1:nqamoyen`) %>% 
  select(`nqamoyen`) %>% 
  unnest() %>% 
  mutate(recordId = 1:nrow(.)) %>% 
  #mutate(nqamoyen = stringr::str_remove(.value, 'd1:')) %>%
  mutate(nqamoyen = .value) %>%
  select(recordId, nqamoyen)
}


extraRecordData <- 1:nb_rep %>% 
  parse_extraRecordData

extraRecordData <- extraRecordData %>% mutate(nqamoyen = as.numeric(nqamoyen))

head(extraRecordData)

Warning message:
“`cols` is now required when using unnest().
Please use `cols = c(nqamoyen)`”

recordId,nqamoyen
1,0.00
2,93.86
3,99.98
4,96.75
5,99.98
6,100.00


In [9]:
# Filtre domaine public : 

works <- tot_df %>% 
  filter(grepl('public', rights), as.integer(stringr::str_extract(date, '[0-9]{4}')) < 1946 ) %>% 
  inner_join(extraRecordData %>% filter(nqamoyen > 80), by = "recordId")
  # inner_join(extraRecordData %>% filter(nqamoyen > 85), by = "recordId")

nrow(works)

[1] 84

In [10]:
head(works)

recordId,contributor,creator,date,description,format,identifier,language,publisher,relation,rights,source,subject,title,type,nqamoyen
2,,"Maupassant, Guy de (1850-1893). Auteur du texte",1882,Contient une table des matières -- Avec mode texte,1 vol. (172 p.) : portrait ; in-16 -- Nombre total de vues : 186,https://gallica.bnf.fr/ark:/12148/bpt6k5680846d,fre -- français,H. Kistemaeckers (Bruxelles),Notice du catalogue : http://catalogue.bnf.fr/ark:/12148/cb324319162,domaine public -- public domain,"Bibliothèque nationale de France, département Réserve des livres rares, RES P-Y2-2092",,Mlle Fifi / Guy de Maupassant ; eau-forte par Just,text -- monographie imprimée -- monographie imprimée,93.86
3,,"Maupassant, Guy de (1850-1893). Auteur du texte",1884,Contient une table des matières -- Avec mode texte,III-348 p. ; in-18 -- Nombre total de vues : 362 -- application/epub+zip,https://gallica.bnf.fr/ark:/12148/bpt6k58416576,fre -- français,V. Havard (Paris),Notice du catalogue : http://catalogue.bnf.fr/ark:/12148/cb30910585q,domaine public -- public domain,"Bibliothèque nationale de France, département Littérature et art, 8-Y2-6879",,Miss Harriet / Guy de Maupassant,text -- monographie imprimée -- monographie imprimée,99.98
4,"Normandy, Georges (1882-1946). Éditeur scientifique","Maupassant, Guy de (1850-1893). Auteur du texte",1913,Contient une table des matières -- Avec mode texte,"1 vol. (187 p.) : portr., fac-sim., pl. ; in-16 -- Nombre total de vues : 196",https://gallica.bnf.fr/ark:/12148/bpt6k5801596v,fre -- français,A. Méricant (Paris),Notice du catalogue : http://catalogue.bnf.fr/ark:/12148/cb365719754,domaine public -- public domain,"Bibliothèque nationale de France, département Littérature et art, Z BARRES-22799",,"Anthologie des auteurs modernes. V, Recueil de morceaux choisis / Guy de Maupassant ; précédé d'une étude... par Georges Normandy...",text -- monographie imprimée -- monographie imprimée,96.75
5,,"Maupassant, Guy de (1850-1893). Auteur du texte",1883,[Contes de la bécasse (français)] -- Avec mode texte,"1 vol. (298 p.) ; in-16 -- Nombre total de vues : 313 -- application/epub+zip 3.0 accessible -- Format adaptable de type XML DTBook, 2005-3",https://gallica.bnf.fr/ark:/12148/bpt6k10407406,fre -- français,E. Rouveyre et G. Blond (Paris),Notice d’oeuvre : http://catalogue.bnf.fr/ark:/12148/cb12376516m -- Notice du catalogue : http://catalogue.bnf.fr/ark:/12148/cb324318906,domaine public -- public domain,"Bibliothèque nationale de France, département Réserve des livres rares, RES 8-Z DON-595 (61)",,Contes de la bécasse. 2è édition (2e édition) / Guy de Maupassant,text -- monographie imprimée -- monographie imprimée,99.98
6,,"Maupassant, Guy de (1850-1893). Auteur du texte",1884,Avec mode texte,"1 vol. (145 p.) : fig., pl. en divers états ; in-4 -- Nombre total de vues : 209 -- application/epub+zip 3.0 accessible -- Format adaptable de type XML DTBook, 2005-3",https://gallica.bnf.fr/ark:/12148/bpt6k10407688,fre -- français,E. Monnier (Paris),Notice du catalogue : http://catalogue.bnf.fr/ark:/12148/cb309105088,domaine public -- public domain,"Bibliothèque nationale de France, département Réserve des livres rares, SMITH LESOUEF R-6505",,Clair de lune / Guy de Maupassant,text -- monographie imprimée -- monographie imprimée,100.00
7,,"Zola, Émile (1840-1902). Auteur du texte -- Maupassant, Guy de (1850-1893). Auteur du texte -- Huysmans, Joris-Karl (1848-1907). Auteur du texte -- Céard, Henry (1851-1924). Auteur du texte -- Hennique, Léon (1851-1935). Auteur du texte -- Alexis, Paul (1847-1901). Auteur du texte",1880,Avec mode texte,[6]-295 p. : aquarelles ; in-18 -- Nombre total de vues : 317,https://gallica.bnf.fr/ark:/12148/btv1b86268548,fre -- français,G. Charpentier (Paris),Notice du catalogue : http://catalogue.bnf.fr/ark:/12148/cb33607241n,domaine public -- public domain,"Bibliothèque nationale de France, département Réserve des livres rares, RES 8-Z DON-594 (376)",,"Les soirées de Médan / Émile Zola, Guy de Maup

In [18]:
## Sauvegarde pour réutilisation plus tard :

write.csv(works, "../Metadonnees.csv")

In [11]:
library(rvest)

i = 1

# Fonction pour récupérer le texte brut (OCR document par document)
texteBrut <- function(i){
  tem <- works %>% 
    filter(recordId == i)
  contenu <-  tem %>% 
    pull(identifier) %>%
    paste0(., '.texteBrut') %>%
    read_html() %>%
    html_nodes('p') %>%
    html_text() %>%
    paste0(collapse = "\n") %>%
    stringr::str_split("En savoir plus sur l'OCR", n = 2, simplify = TRUE)
  readr::write_csv(data_frame(identifier = tem$identifier, entete = contenu[1,1],
                              texteBrut = contenu[1,2],
                              recordId = tem$recordId), paste0('docs_ocr/', tem$recordId, '.csv')) 
                                                                # Choix du chemin de destination des csv
}

texteBrut(84) # Nombre de documents océrisés à récupérer. Cf. sortie nrow(works)


Attaching package: ‘rvest’

The following object is masked from ‘package:purrr’:

    pluck

Warning message:
“`data_frame()` is deprecated as of tibble 1.1.0.
Please use `tibble()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”

---

   - Export document par document en csv avec : ark, entete, texteOCR

Prend du temps. On peut aller dans le dossier de destination pour voir l'avancée.

In [12]:
deja_ok <- list.files('data/docs_ocr/') %>% 
  stringr::str_remove('\\.csv') %>% 
  as.integer

todo <- unique(works$recordId)  %>% 
  .[!(. %in% deja_ok)]
todo <- todo %>% sort()

todo <- todo %>% purrr::map(purrr::possibly(texteBrut, otherwise = NULL))

## Fin du notebook :

   - Récupération des OCR : FAIT.
   - Passage aux analyses et visualisations.